In [1]:
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits
from specutils import Spectrum1D
from scipy.interpolate import interp1d

from glob import glob

# Importing Data
The variable cubes should be a list of paths for filenames for the end products of the jwst pipeline. Either by band or b channel

In [5]:
cubes = sorted(glob('Phoenix_cal/stage3mod/*_s3d.fits'))
corrx, corry = np.genfromtxt('miri_sensitivity_correction.txt')
print(cubes)

['Phoenix_cal/stage3mod/Level3_ch1-long_s3d.fits', 'Phoenix_cal/stage3mod/Level3_ch1-medium_s3d.fits', 'Phoenix_cal/stage3mod/Level3_ch1-short_s3d.fits', 'Phoenix_cal/stage3mod/Level3_ch2-long_s3d.fits', 'Phoenix_cal/stage3mod/Level3_ch2-medium_s3d.fits', 'Phoenix_cal/stage3mod/Level3_ch2-short_s3d.fits', 'Phoenix_cal/stage3mod/Level3_ch3-long_s3d.fits', 'Phoenix_cal/stage3mod/Level3_ch3-medium_s3d.fits', 'Phoenix_cal/stage3mod/Level3_ch3-short_s3d.fits', 'Phoenix_cal/stage3mod/Level3_ch4-long_s3d.fits', 'Phoenix_cal/stage3mod/Level3_ch4-medium_s3d.fits', 'Phoenix_cal/stage3mod/Level3_ch4-short_s3d.fits']
['Phoenix_cal/poststage3/Level3_ch1-long_s3d_weightedbackground.fits', 'Phoenix_cal/poststage3/Level3_ch1-medium_s3d_weightedbackground.fits', 'Phoenix_cal/poststage3/Level3_ch1-short_s3d_weightedbackground.fits', 'Phoenix_cal/poststage3/Level3_ch2-long_s3d_weightedbackground.fits', 'Phoenix_cal/poststage3/Level3_ch2-medium_s3d_weightedbackground.fits', 'Phoenix_cal/poststage3/Level3_

# Correcting
This notebook should increase the general time and wavelength dependent correction factor implemented by the jwst pipeline due a sensitivity drop. While not perfect we have modeled this correction factor as a smooth polynomial (based on the jwst correction) that is scaled with a power law to account for the time dependence. The wavelength dependent drop is fairly constant source to source so the only parameter that needs changing is time dependent factor.

We determined the correction factor for Phoenix by bringing the jwst data within the error bars of WISE data from the same source. With the limited data set correction factors should be determined source to source. 

In [6]:
##### Change based on observation date #########
corr_factor = 0.75 ## 27-07-2023 (Phoenix)

for i, name in enumerate(cubes):
    hdul = fits.open(name)
    hdr = hdul[1].header
    wav = (np.arange(hdr['NAXIS3']) + hdr['CRPIX3'] - 1) * hdr['CDELT3'] + hdr['CRVAL3']
    corr_interp = interp1d(corrx, corry)(wav)
    
    # uncommenting the lines below to show the old and corrected spectra for an individual spaxel
    for j in range(hdul[1].data.shape[1]):
        for k in range(hdul[1].data.shape[2]):
            hdul[2].data[:, j, k] = hdul[2].data[:, j, k]*corr_interp**corr_factor

    hdul.writeto(name[:-5] + '_corr_nostripe.fits', overwrite=True)